In [0]:
%tensorflow_version 2.x

In [137]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [138]:
cd /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/SLSTM/hparams/3

/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/SLSTM/hparams/3


# Automated Conversion of *.py file

In [0]:
# # -*- coding: utf-8 -*-
# """slstm_with_fasttext_with_skopt.ipynb

# Automatically generated by Colaboratory.

# Original file is located at
#     https://colab.research.google.com/drive/1idQaa6eyIsPYYY2jbxVvAd7KP7Um_e8v
# """

# # %tensorflow_version 2.x

# # import tensorflow.compat.v1 as tf
# # tf.disable_v2_behavior()

# # from google.colab import drive
# # drive.mount('/content/drive')

# path='/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/SLSTM/parsed_data/from_fasttext/data_set'
# vector_path = '/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/SLSTM/parsed_data/from_fasttext/fasttext_vectors'
# # run from lahiru1st@gmail.com
# # path='/content/drive/My Drive/UNI/FYP/Sentiment Analysis/supportive/S-LSTM/classfication/parsed_data/from_fasttext/data_set'
# # vector_path = '/content/drive/My Drive/UNI/FYP/Sentiment Analysis/supportive/S-LSTM/classfication/parsed_data/from_fasttext/fasttext_vectors'

# """# Imports

# ## Standard Imports
# """

# from __future__ import print_function
# # import six.moves.cPickle as pickle
# # from collections import OrderedDict
# import sys
# import time
# import numpy as np
# import tensorflow as tf
# # import read_data
# # from random import shuffle
# # import random
# import pickle
# # import math
# from keras.preprocessing.sequence import pad_sequences
# from sklearn.metrics import confusion_matrix
# # from general_utils import Progbar
# # import tensorflow.contrib.slim as slim
# # from sst_config import Config

# """## config"""

# class Config(object):
#     vocab_size=15000
#     max_grad_norm = 5
#     init_scale = 0.05
#     hidden_size = 300
#     lr_decay = 0.95
#     valid_portion=0.0
#     batch_size=5
#     keep_prob = 0.5
#     #0.05
#     learning_rate = 0.002
#     max_epoch =2
#     # max_max_epoch =40
#     max_max_epoch = 30
#     num_label=5
#     attention_iteration=3
#     random_initialize=False
#     embedding_trainable=True
#     l2_beta=0.0

# """## Progbar"""

# import time
# import sys
# import logging
# import numpy as np


# class Progbar(object):
#     """Progbar class copied from keras (https://github.com/fchollet/keras/)

#     Displays a progress bar.
#     Small edit : added strict arg to update
#     # Arguments
#         target: Total number of steps expected.
#         interval: Minimum visual progress update interval (in seconds).
#     """

#     def __init__(self, target, width=30, verbose=0):
#         self.width = width
#         self.target = target
#         self.sum_values = {}
#         self.unique_values = []
#         self.start = time.time()
#         self.total_width = 0
#         self.seen_so_far = 0
#         self.verbose = verbose

#     def update(self, current, values=[], exact=[], strict=[]):
#         """
#         Updates the progress bar.
#         # Arguments
#             current: Index of current step.
#             values: List of tuples (name, value_for_last_step).
#                 The progress bar will display averages for these values.
#             exact: List of tuples (name, value_for_last_step).
#                 The progress bar will display these values directly.
#         """

#         for k, v in values:
#             if k not in self.sum_values:
#                 self.sum_values[k] = [v * (current - self.seen_so_far),
#                                       current - self.seen_so_far]
#                 self.unique_values.append(k)
#             else:
#                 self.sum_values[k][0] += v * (current - self.seen_so_far)
#                 self.sum_values[k][1] += (current - self.seen_so_far)
#         for k, v in exact:
#             if k not in self.sum_values:
#                 self.unique_values.append(k)
#             self.sum_values[k] = [v, 1]

#         for k, v in strict:
#             if k not in self.sum_values:
#                 self.unique_values.append(k)
#             self.sum_values[k] = v

#         self.seen_so_far = current

#         now = time.time()
#         if self.verbose == 1:
#             prev_total_width = self.total_width
#             sys.stdout.write("\b" * prev_total_width)
#             sys.stdout.write("\r")

#             numdigits = int(np.floor(np.log10(self.target))) + 1
#             barstr = '%%%dd/%%%dd [' % (numdigits, numdigits)
#             bar = barstr % (current, self.target)
#             prog = float(current)/self.target
#             prog_width = int(self.width*prog)
#             if prog_width > 0:
#                 bar += ('='*(prog_width-1))
#                 if current < self.target:
#                     bar += '>'
#                 else:
#                     bar += '='
#             bar += ('.'*(self.width-prog_width))
#             bar += ']'
#             sys.stdout.write(bar)
#             self.total_width = len(bar)

#             if current:
#                 time_per_unit = (now - self.start) / current
#             else:
#                 time_per_unit = 0
#             eta = time_per_unit*(self.target - current)
#             info = ''
#             if current < self.target:
#                 info += ' - ETA: %ds' % eta
#             else:
#                 info += ' - %ds' % (now - self.start)
#             for k in self.unique_values:
#                 if type(self.sum_values[k]) is list:
#                     info += ' - %s: %.4f' % (k,
#                         self.sum_values[k][0] / max(1, self.sum_values[k][1]))
#                 else:
#                     info += ' - %s: %s' % (k, self.sum_values[k])

#             self.total_width += len(info)
#             if prev_total_width > self.total_width:
#                 info += ((prev_total_width-self.total_width) * " ")

#             sys.stdout.write(info)
#             sys.stdout.flush()

#             if current >= self.target:
#                 sys.stdout.write("\n")

#         if self.verbose == 2:
#             if current >= self.target:
#                 info = '%ds' % (now - self.start)
#                 for k in self.unique_values:
#                     info += ' - %s: %.4f' % (k,
#                         self.sum_values[k][0] / max(1, self.sum_values[k][1]))
#                 sys.stdout.write(info + "\n")

#     def add(self, n, values=[]):
#         self.update(self.seen_so_far+n, values)

# """## Read Data"""

# from __future__ import print_function
# from six.moves import xrange
# import six.moves.cPickle as pickle
# import gzip
# import os
# import numpy

# def generate_matrix(seqs, maxlen, lengths):
#     n_samples = len(seqs)
#     x= numpy.zeros((n_samples, maxlen)).astype('int64')

#     for idx, s in enumerate(seqs):
#         if lengths[idx]>= maxlen:
#             s=s[:maxlen]
#         x[idx, :lengths[idx]] = s
#     return x

# def prepare_data(seqs, labels):
#     lengths = [len(s) for s in seqs]
#     labels = numpy.array(labels).astype('int32')
#     return [numpy.array(seqs), labels, numpy.array(lengths).astype('int32')]

# def remove_unk(x, n_words):
#     return [[1 if w >= n_words else w for w in sen] for sen in x] 

# def load_data(path, n_words):
#     with open(path, 'rb') as f:
#         dataset_x, dataset_label= pickle.load(f)
#         train_set_x, train_set_y = dataset_x[0], dataset_label[0]
#         # valid_set_x, valid_set_y =dataset_x[1], dataset_label[1]
#         test_set_x, test_set_y = dataset_x[1], dataset_label[1]
#     #remove unknown words
#     train_set_x = remove_unk(train_set_x, n_words)
#     # valid_set_x = remove_unk(valid_set_x, n_words)
#     test_set_x = remove_unk(test_set_x, n_words)

#     return [train_set_x, train_set_y],[test_set_x, test_set_y]

# """# Implementation

# ## LSTM Layer
# """

# def lstm_layer(initial_hidden_states, config, keep_prob, mask):
#     with tf.compat.v1.variable_scope('forward'):
#         fw_lstm = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(config.hidden_size, forget_bias=0.0)
#         fw_lstm = tf.contrib.rnn.DropoutWrapper(fw_lstm, output_keep_prob=keep_prob)

#     with tf.compat.v1.variable_scope('backward'):
#         bw_lstm = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(config.hidden_size, forget_bias=0.0)
#         bw_lstm = tf.contrib.rnn.DropoutWrapper(bw_lstm, output_keep_prob=keep_prob)

#     #bidirectional rnn
#     with tf.compat.v1.variable_scope('bilstm'):
#         lstm_output=tf.compat.v1.nn.bidirectional_dynamic_rnn(fw_lstm, bw_lstm, inputs=initial_hidden_states, sequence_length=mask, time_major=False, dtype=tf.float32)
#         lstm_output=tf.concat(lstm_output[0], 2)

#     return lstm_output

# """## Classifier"""

# class Classifer(object):

#     def get_hidden_states_before(self, hidden_states, step, shape, hidden_size):
#         #padding zeros
#         padding=tf.zeros((shape[0], step, hidden_size), dtype=tf.float32)
#         #remove last steps
#         displaced_hidden_states=hidden_states[:,:-step,:]
#         #concat padding
#         return tf.concat([padding, displaced_hidden_states], axis=1)
#         #return tf.cond(step<=shape[1], lambda: tf.concat([padding, displaced_hidden_states], axis=1), lambda: tf.zeros((shape[0], shape[1], self.config.hidden_size_sum), dtype=tf.float32))

#     def get_hidden_states_after(self, hidden_states, step, shape, hidden_size):
#         #padding zeros
#         padding=tf.zeros((shape[0], step, hidden_size), dtype=tf.float32)
#         #remove last steps
#         displaced_hidden_states=hidden_states[:,step:,:]
#         #concat padding
#         return tf.concat([displaced_hidden_states, padding], axis=1)
#         #return tf.cond(step<=shape[1], lambda: tf.concat([displaced_hidden_states, padding], axis=1), lambda: tf.zeros((shape[0], shape[1], self.config.hidden_size_sum), dtype=tf.float32))

#     def sum_together(self, l):
#         combined_state=None
#         for tensor in l:
#             if combined_state==None:
#                 combined_state=tensor
#             else:
#                 combined_state=combined_state+tensor
#         return combined_state
    
#     def slstm_cell(self, name_scope_name, hidden_size, lengths, initial_hidden_states, initial_cell_states, num_layers):
#         with tf.compat.v1.name_scope(name_scope_name):
#             #Word parameters 
#             #forget gate for left 
#             with tf.compat.v1.name_scope("f1_gate"):
#                 #current
#                 Wxf1 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
#                 #left right
#                 Whf1 = tf.Variable(tf.random.normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
#                 #initial state
#                 Wif1 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wif")
#                 #dummy node
#                 Wdf1 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdf")
#             #forget gate for right 
#             with tf.compat.v1.name_scope("f2_gate"):
#                 Wxf2 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
#                 Whf2 = tf.Variable(tf.random.normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
#                 Wif2 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wif")
#                 Wdf2 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdf")
#             #forget gate for inital states     
#             with tf.compat.v1.name_scope("f3_gate"):
#                 Wxf3 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
#                 Whf3 = tf.Variable(tf.random.normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
#                 Wif3 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wif")
#                 Wdf3 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdf")
#             #forget gate for dummy states     
#             with tf.compat.v1.name_scope("f4_gate"):
#                 Wxf4 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
#                 Whf4 = tf.Variable(tf.random.normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
#                 Wif4 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wif")
#                 Wdf4 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdf")
#             #input gate for current state     
#             with tf.compat.v1.name_scope("i_gate"):
#                 Wxi = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxi")
#                 Whi = tf.Variable(tf.random.normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whi")
#                 Wii = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wii")
#                 Wdi = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdi")
#             #input gate for output gate
#             with tf.compat.v1.name_scope("o_gate"):
#                 Wxo = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxo")
#                 Who = tf.Variable(tf.random.normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Who")
#                 Wio = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wio")
#                 Wdo = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdo")
#             #bias for the gates    
#             with tf.compat.v1.name_scope("biases"):
#                 bi = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bi")
#                 bo = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bo")
#                 bf1 = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bf1")
#                 bf2 = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bf2")
#                 bf3 = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bf3")
#                 bf4 = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bf4")

#             #dummy node gated attention parameters
#             #input gate for dummy state
#             with tf.compat.v1.name_scope("gated_d_gate"):
#                 gated_Wxd = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
#                 gated_Whd = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
#             #output gate
#             with tf.compat.v1.name_scope("gated_o_gate"):
#                 gated_Wxo = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxo")
#                 gated_Who = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Who")
#             #forget gate for states of word
#             with tf.compat.v1.name_scope("gated_f_gate"):
#                 gated_Wxf = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxo")
#                 gated_Whf = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Who")
#             #biases
#             with tf.compat.v1.name_scope("gated_biases"):
#                 gated_bd = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bi")
#                 gated_bo = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bo")
#                 gated_bf = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bo")

#         #filters for attention        
#         mask_softmax_score=tf.cast(tf.sequence_mask(lengths), tf.float32)*1e25-1e25
#         mask_softmax_score_expanded=tf.expand_dims(mask_softmax_score, axis=2)               
#         #filter invalid steps
#         sequence_mask=tf.expand_dims(tf.cast(tf.sequence_mask(lengths), tf.float32),axis=2)
#         #filter embedding states
#         initial_hidden_states=initial_hidden_states*sequence_mask
#         initial_cell_states=initial_cell_states*sequence_mask
#         #record shape of the batch
#         shape=tf.shape(input=initial_hidden_states)
        
#         #initial embedding states
#         embedding_hidden_state=tf.reshape(initial_hidden_states, [-1, hidden_size])      
#         embedding_cell_state=tf.reshape(initial_cell_states, [-1, hidden_size])

#         #randomly initialize the states
#         if config.random_initialize:
#             initial_hidden_states=tf.random.uniform(shape, minval=-0.05, maxval=0.05, dtype=tf.float32, seed=None, name=None)
#             initial_cell_states=tf.random.uniform(shape, minval=-0.05, maxval=0.05, dtype=tf.float32, seed=None, name=None)
#             #filter it
#             initial_hidden_states=initial_hidden_states*sequence_mask
#             initial_cell_states=initial_cell_states*sequence_mask

#         #inital dummy node states
#         dummynode_hidden_states=tf.reduce_mean(input_tensor=initial_hidden_states, axis=1)
#         dummynode_cell_states=tf.reduce_mean(input_tensor=initial_cell_states, axis=1)

#         for i in range(num_layers):
#             #update dummy node states
#             #average states
#             combined_word_hidden_state=tf.reduce_mean(input_tensor=initial_hidden_states, axis=1)
#             reshaped_hidden_output=tf.reshape(initial_hidden_states, [-1, hidden_size])
#             #copy dummy states for computing forget gate
#             transformed_dummynode_hidden_states=tf.reshape(tf.tile(tf.expand_dims(dummynode_hidden_states, axis=1), [1, shape[1],1]), [-1, hidden_size])
#             #input gate
#             gated_d_t = tf.nn.sigmoid(
#                 tf.matmul(dummynode_hidden_states, gated_Wxd) + tf.matmul(combined_word_hidden_state, gated_Whd) + gated_bd
#             )
#             #output gate
#             gated_o_t = tf.nn.sigmoid(
#                 tf.matmul(dummynode_hidden_states, gated_Wxo) + tf.matmul(combined_word_hidden_state, gated_Who) + gated_bo
#             )
#             #forget gate for hidden states
#             gated_f_t = tf.nn.sigmoid(
#                 tf.matmul(transformed_dummynode_hidden_states, gated_Wxf) + tf.matmul(reshaped_hidden_output, gated_Whf) + gated_bf
#             )

#             #softmax on each hidden dimension 
#             reshaped_gated_f_t=tf.reshape(gated_f_t, [shape[0], shape[1], hidden_size])+ mask_softmax_score_expanded
#             gated_softmax_scores=tf.nn.softmax(tf.concat([reshaped_gated_f_t, tf.expand_dims(gated_d_t, axis=1)], axis=1), axis=1)
#             #split the softmax scores
#             new_reshaped_gated_f_t=gated_softmax_scores[:,:shape[1],:]
#             new_gated_d_t=gated_softmax_scores[:,shape[1]:,:]
#             #new dummy states
#             dummy_c_t=tf.reduce_sum(input_tensor=new_reshaped_gated_f_t * initial_cell_states, axis=1) + tf.squeeze(new_gated_d_t, axis=1)*dummynode_cell_states
#             dummy_h_t=gated_o_t * tf.nn.tanh(dummy_c_t)

#             #update word node states
#             #get states before
#             initial_hidden_states_before=[tf.reshape(self.get_hidden_states_before(initial_hidden_states, step+1, shape, hidden_size), [-1, hidden_size]) for step in range(self.config.step)]
#             initial_hidden_states_before=self.sum_together(initial_hidden_states_before)
#             initial_hidden_states_after= [tf.reshape(self.get_hidden_states_after(initial_hidden_states, step+1, shape, hidden_size), [-1, hidden_size]) for step in range(self.config.step)]
#             initial_hidden_states_after=self.sum_together(initial_hidden_states_after)
#             #get states after
#             initial_cell_states_before=[tf.reshape(self.get_hidden_states_before(initial_cell_states, step+1, shape, hidden_size), [-1, hidden_size]) for step in range(self.config.step)]
#             initial_cell_states_before=self.sum_together(initial_cell_states_before)
#             initial_cell_states_after=[tf.reshape(self.get_hidden_states_after(initial_cell_states, step+1, shape, hidden_size), [-1, hidden_size]) for step in range(self.config.step)]
#             initial_cell_states_after=self.sum_together(initial_cell_states_after)
            
#             #reshape for matmul
#             initial_hidden_states=tf.reshape(initial_hidden_states, [-1, hidden_size])
#             initial_cell_states=tf.reshape(initial_cell_states, [-1, hidden_size])

#             #concat before and after hidden states
#             concat_before_after=tf.concat([initial_hidden_states_before, initial_hidden_states_after], axis=1)

#             #copy dummy node states 
#             transformed_dummynode_hidden_states=tf.reshape(tf.tile(tf.expand_dims(dummynode_hidden_states, axis=1), [1, shape[1],1]), [-1, hidden_size])
#             transformed_dummynode_cell_states=tf.reshape(tf.tile(tf.expand_dims(dummynode_cell_states, axis=1), [1, shape[1],1]), [-1, hidden_size])

#             f1_t = tf.nn.sigmoid(
#                 tf.matmul(initial_hidden_states, Wxf1) + tf.matmul(concat_before_after, Whf1) + 
#                 tf.matmul(embedding_hidden_state, Wif1) + tf.matmul(transformed_dummynode_hidden_states, Wdf1)+ bf1
#             )

#             f2_t = tf.nn.sigmoid(
#                 tf.matmul(initial_hidden_states, Wxf2) + tf.matmul(concat_before_after, Whf2) + 
#                 tf.matmul(embedding_hidden_state, Wif2) + tf.matmul(transformed_dummynode_hidden_states, Wdf2)+ bf2
#             )

#             f3_t = tf.nn.sigmoid(
#                 tf.matmul(initial_hidden_states, Wxf3) + tf.matmul(concat_before_after, Whf3) + 
#                 tf.matmul(embedding_hidden_state, Wif3) + tf.matmul(transformed_dummynode_hidden_states, Wdf3) + bf3
#             )

#             f4_t = tf.nn.sigmoid(
#                 tf.matmul(initial_hidden_states, Wxf4) + tf.matmul(concat_before_after, Whf4) + 
#                 tf.matmul(embedding_hidden_state, Wif4) + tf.matmul(transformed_dummynode_hidden_states, Wdf4) + bf4
#             )
            
#             i_t = tf.nn.sigmoid(
#                 tf.matmul(initial_hidden_states, Wxi) + tf.matmul(concat_before_after, Whi) + 
#                 tf.matmul(embedding_hidden_state, Wii) + tf.matmul(transformed_dummynode_hidden_states, Wdi)+ bi
#             )
            
#             o_t = tf.nn.sigmoid(
#                 tf.matmul(initial_hidden_states, Wxo) + tf.matmul(concat_before_after, Who) + 
#                 tf.matmul(embedding_hidden_state, Wio) + tf.matmul(transformed_dummynode_hidden_states, Wdo) + bo
#             )
            
#             f1_t, f2_t, f3_t, f4_t, i_t=tf.expand_dims(f1_t, axis=1), tf.expand_dims(f2_t, axis=1),tf.expand_dims(f3_t, axis=1), tf.expand_dims(f4_t, axis=1), tf.expand_dims(i_t, axis=1)


#             five_gates=tf.concat([f1_t, f2_t, f3_t, f4_t,i_t], axis=1)
#             five_gates=tf.nn.softmax(five_gates, axis=1)
#             f1_t,f2_t,f3_t, f4_t,i_t= tf.split(five_gates, num_or_size_splits=5, axis=1)
            
#             f1_t, f2_t, f3_t, f4_t, i_t=tf.squeeze(f1_t, axis=1), tf.squeeze(f2_t, axis=1),tf.squeeze(f3_t, axis=1), tf.squeeze(f4_t, axis=1),tf.squeeze(i_t, axis=1)

#             c_t = (f1_t * initial_cell_states_before) + (f2_t * initial_cell_states_after)+(f3_t * embedding_cell_state)+ (f4_t * transformed_dummynode_cell_states)+ (i_t * initial_cell_states)
            
#             h_t = o_t * tf.nn.tanh(c_t)

#             #update states
#             initial_hidden_states=tf.reshape(h_t, [shape[0], shape[1], hidden_size])
#             initial_cell_states=tf.reshape(c_t, [shape[0], shape[1], hidden_size])
#             initial_hidden_states=initial_hidden_states*sequence_mask
#             initial_cell_states=initial_cell_states*sequence_mask

#             dummynode_hidden_states=dummy_h_t
#             dummynode_cell_states=dummy_c_t

#         initial_hidden_states = tf.nn.dropout(initial_hidden_states,1 - (self.dropout))
#         initial_cell_states = tf.nn.dropout(initial_cell_states, 1 - (self.dropout))

#         return initial_hidden_states, initial_cell_states, dummynode_hidden_states



#     def __init__(self, config, session):
#         #inputs: features, mask, keep_prob, labels
#         self.input_data = tf.compat.v1.placeholder(tf.int32, [None, None], name="inputs")
#         self.labels=tf.compat.v1.placeholder(tf.int64, [None,], name="labels")
#         self.mask=tf.compat.v1.placeholder(tf.int32, [None,], name="mask")
#         self.dropout=self.keep_prob=keep_prob=tf.compat.v1.placeholder(tf.float32, name="keep_prob")
#         self.config=config
#         shape=tf.shape(input=self.input_data)
#         #if sys.argv[4]=='lstm':
#         #    self.dummy_input = tf.placeholder(tf.float32, [None, None], name="dummy")
#         #embedding
#         self.embedding=embedding = tf.Variable(tf.random.normal([config.vocab_size, config.hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="embedding", trainable=config.embedding_trainable)
#         #apply embedding
#         initial_hidden_states=tf.nn.embedding_lookup(params=embedding, ids=self.input_data)
#         initial_cell_states=tf.identity(initial_hidden_states)

#         initial_hidden_states = tf.nn.dropout(initial_hidden_states,1 - (keep_prob))
#         initial_cell_states = tf.nn.dropout(initial_cell_states, 1 - (keep_prob))

#         #create layers 
#         if argument4=='slstm':
#             new_hidden_states,new_cell_state, dummynode_hidden_states=self.slstm_cell("word_slstm", config.hidden_size,self.mask, initial_hidden_states, initial_cell_states, config.layer)
            
#             softmax_w = tf.Variable(tf.random.normal([2*config.hidden_size, config.num_label], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_w")
#             softmax_b = tf.Variable(tf.random.normal([config.num_label], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_b")
#             #representation=dummynode_hidden_states
#             representation=tf.reduce_mean(input_tensor=tf.concat([new_hidden_states, tf.expand_dims(dummynode_hidden_states, axis=1)], axis=1), axis=1)
            
#             softmax_w2 = tf.Variable(tf.random.normal([config.hidden_size, 2*config.hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_w2")
#             softmax_b2 = tf.Variable(tf.random.normal([2*config.hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_b2")
#             representation=tf.nn.tanh(tf.matmul(representation, softmax_w2)+softmax_b2)

#         elif argument4=='lstm':
#             initial_hidden_states=lstm_layer(initial_hidden_states,config, self.keep_prob, self.mask)
#             softmax_w = tf.Variable(tf.random.normal([2*config.hidden_size, config.num_label], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_w")
#             softmax_b = tf.Variable(tf.random.normal([config.num_label], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_b")
#             representation=tf.reduce_sum(input_tensor=initial_hidden_states,axis=1)
#             config.hidden_size_sum=2*config.hidden_size
#         elif argument4=='cnn':
#             initial_hidden_states=tf.reshape(initial_hidden_states, [-1, 700, config.hidden_size])            
#             initial_hidden_states = tf.expand_dims(initial_hidden_states, -1)
#             pooled_outputs = []
#             for i, filter_size in enumerate([3]):
#                 with tf.compat.v1.name_scope("conv-maxpool-%s" % filter_size):
#                     # Convolution Layer
#                     filter_shape = [filter_size, config.hidden_size, 1, config.hidden_size]
#                     W = tf.Variable(tf.random.truncated_normal(filter_shape, stddev=0.1), name="W")
#                     b = tf.Variable(tf.constant(0.1, shape=[config.hidden_size]), name="b")

#                     W2 = tf.Variable(tf.random.truncated_normal(filter_shape, stddev=0.1), name="W2")
#                     b2 = tf.Variable(tf.constant(0.1, shape=[config.hidden_size]), name="b2")

#                     W3 = tf.Variable(tf.random.truncated_normal(filter_shape, stddev=0.1), name="W3")
#                     b3 = tf.Variable(tf.constant(0.1, shape=[config.hidden_size]), name="b3")

#                     W4 = tf.Variable(tf.random.truncated_normal(filter_shape, stddev=0.1), name="W4")
#                     b4 = tf.Variable(tf.constant(0.1, shape=[config.hidden_size]), name="b4")

#                     conv = tf.nn.conv2d(
#                         input=initial_hidden_states,
#                         filters=W,
#                         strides=[1, 1, 1, 1],
#                         padding="VALID",
#                         name="conv")
#                     h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
#                     print(h.get_shape())
#                     h=tf.transpose(a=h, perm=[0,1,3,2])
#                     # Apply nonlinearity


#                     conv2 = tf.nn.conv2d(
#                         input=h,
#                         filters=W2,
#                         strides=[1, 1, 1, 1],
#                         padding="VALID",
#                         name="conv2")
#                     h2 = tf.nn.relu(tf.nn.bias_add(conv2, b2), name="relu2")
#                     print(h2.get_shape())
#                     h2=tf.transpose(a=h2, perm=[0,1,3,2])

#                     conv3 = tf.nn.conv2d(
#                         input=h2,
#                         filters=W3,
#                         strides=[1, 1, 1, 1],
#                         padding="VALID",
#                         name="conv3")  
#                     h3 = tf.nn.relu(tf.nn.bias_add(conv3, b3), name="relu3")
#                     print(h3.get_shape())

#                     # Max-pooling over the outputs
#                     pooled = tf.nn.max_pool2d(
#                         input=h3,
#                         ksize=[1, 700 - 3*filter_size + 3, 1, 1],
#                         strides=[1, 1, 1, 1],
#                         padding='VALID',
#                         name="pool")
#                     pooled_outputs.append(pooled)
#             # Combine all the pooled features
#             num_filters_total = 1 * config.hidden_size
#             self.h_pool = tf.concat(pooled_outputs, axis=3)
#             representation = tf.reshape(self.h_pool, [-1, num_filters_total])

#             softmax_w = tf.Variable(tf.random.normal([2*config.hidden_size, config.num_label], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_w")
#             softmax_b = tf.Variable(tf.random.normal([config.num_label], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_b")

#             softmax_w2 = tf.Variable(tf.random.normal([config.hidden_size, 2*config.hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_w2")
#             softmax_b2 = tf.Variable(tf.random.normal([2*config.hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_b2")
#             representation=tf.nn.tanh(tf.matmul(representation, softmax_w2)+softmax_b2)
#         else:
#             print("Invalid model")
#             exit(1)
        
#         self.logits=logits = tf.matmul(representation, softmax_w) + softmax_b
#         self.to_print=tf.nn.softmax(logits)
#         #operators for prediction
#         self.prediction=prediction=tf.argmax(input=logits,axis=1)
#         correct_prediction = tf.equal(prediction, self.labels)
#         self.accuracy = tf.reduce_sum(input_tensor=tf.cast(correct_prediction, tf.float32))
        
#         #cross entropy loss
#         loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.labels, logits=logits)
#         self.cost=cost=tf.reduce_mean(input_tensor=loss)+ config.l2_beta*tf.nn.l2_loss(embedding)

#         #designate training variables
#         tvars=tf.compat.v1.trainable_variables()
#         self.lr = tf.Variable(0.0, trainable=False)
#         grads=tf.gradients(ys=cost, xs=tvars)
#         grads, _ = tf.clip_by_global_norm(grads,config.max_grad_norm)
#         self.grads=grads
#         optimizer = tf.compat.v1.train.AdamOptimizer(config.learning_rate)        
#         self.train_op = optimizer.apply_gradients(zip(grads, tvars))

#     #assign value to learning rate
#     def assign_lr(self, session, lr_value):
#         session.run(tf.compat.v1.assign(self.lr, lr_value))

# """## get_minibatches_idx()"""

# def get_minibatches_idx(n, batch_size, shuffle=True):
#     idx_list = np.arange(n, dtype="int32")

#     if shuffle:
#         np.random.shuffle(idx_list)

#     minibatches = []
#     minibatch_start = 0
#     for i in range(n // batch_size):
#         minibatches.append(idx_list[minibatch_start:
#                                     minibatch_start + batch_size])
#         minibatch_start += batch_size
#     if (minibatch_start != n):
#         # Make a minibatch out of what is left
#         minibatches.append(idx_list[minibatch_start:])
#     return minibatches

# """## run_epoch"""

# def run_epoch(session, config, model, data, eval_op, keep_prob, is_training):
#     n_samples = len(data[0])
#     print("Running %d samples:"%(n_samples))  
#     minibatches = get_minibatches_idx(n_samples, config.batch_size, shuffle=False)

#     predictions = []
#     correct = 0.
#     total = 0
#     total_cost=0
#     prog = Progbar(target=len(minibatches))
#     #dummynode_hidden_states_collector=np.array([[0]*config.hidden_size])

#     to_print_total=np.array([[0]*2])
#     for i, inds in enumerate(minibatches):
#         x = data[0][inds]
#         if argument4=='cnn':
#             x=pad_sequences(x, maxlen=700, dtype='int32',padding='post', truncating='post', value=0.)
#         else:
#             x=pad_sequences(x, maxlen=None, dtype='int32',padding='post', truncating='post', value=0.)
#         y = data[1][inds]
#         mask = data[2][inds]



#         count, _, cost, to_print,prediction= \
#         session.run([model.accuracy, eval_op,model.cost, model.to_print,model.prediction],\
#             {model.input_data: x, model.labels: y, model.mask:mask, model.keep_prob:keep_prob}) 
        

#         if not is_training:
#             to_print_total=np.concatenate((to_print_total, to_print),axis=0)
#         correct += count 
#         total += len(inds)
#         total_cost+=cost
#         predictions.extend(prediction.tolist())
#         prog.update(i + 1, [("train loss", cost)])
#     #if not is_training:
#     #    print(to_print_total[:, 0].tolist())
#     #    print(data[1].tolist())
#     #    print(data[2].tolist())

#     actual = data[1]

#     TN, FP, FN, TP = confusion_matrix(actual, predictions).ravel()

#     precision = TP / (TP + FP)
#     recall = TP / (TP + FN)
#     f1 = 2 * precision * recall / (precision + recall)

#     print("Total loss:")
#     print(total_cost)

#     accuracy = correct/total

#     return accuracy,precision,recall,f1, actual, predictions

# """## train_test_model"""

# def train_test_model(config, i, session, model, train_dataset,test_dataset):
#     #compute lr_decay
#     lr_decay = config.lr_decay ** max(i - config.max_epoch, 0.0)
#     #update learning rate
#     model.assign_lr(session, config.learning_rate * lr_decay)

#     #training            
#     print("Epoch: %d Learning rate: %.5f" % (i + 1, session.run(model.lr)))
#     start_time = time.time()
#     train_acc, train_precision, train_recall, train_f1,actual,predictions = run_epoch(session, config, model, train_dataset, model.train_op, config.keep_prob, True)
#     print("Training Accuracy = %.4f, time = %.3f seconds\n"%(train_acc, time.time()-start_time))

#     # #valid 
#     # valid_acc = run_epoch(session, config, model, valid_dataset, tf.no_op(),1, False)
#     # print("Valid Accuracy = %.4f\n" % valid_acc)

#     #testing
#     start_time = time.time()
#     test_acc,test_precision, test_recall, test_f1,test_actual,test_predictions = run_epoch(session, config, model, test_dataset, tf.no_op(),1, False)
#     print("Test Accuracy = %.4f, Test Precision = %.4f, Test Recall = %.4f, Test F1 = %.4f\n" % (test_acc,test_precision,test_recall,test_f1))    
#     print("Time = %.3f seconds\n"%(time.time()-start_time))
#     print('confusion metric : ')
#     print((confusion_matrix(test_actual, test_predictions).ravel()))
#     #return valid_acc, test_acc
#     return test_actual,test_predictions

# """## start_epoches"""

# def start_epoches(config, session,classifier, train_dataset, test_dataset):
#     #record max
#     #max_val_acc=-1
#     #max_test_acc=-1
#     # saver = tf.train.Saver()
#     all_actual = []
#     all_predictions = []

#     for i in range(config.max_max_epoch):
#       test_actual,test_predictions = train_test_model(config, i, session, classifier, train_dataset, test_dataset)
      
#       all_actual.extend(test_actual)
#       all_predictions.extend(test_predictions)


#       # if (i % 1 == 0 and i != 0):
#       #   save_path = saver.save(session, "/content/drive/My Drive/University/FYP/Sentiment Analysis/supportive/S-LSTM/Trained_Model/slstm_models/pretrained_lstm.ckpt", global_step=i)
#       #     print("saved to %s" % save_path)

#     TN, FP, FN, TP = confusion_matrix(all_actual, all_predictions).ravel()

#     precision = TP / (TP + FP)
#     recall = TP / (TP + FN)
#     f1 = 2 * precision * recall / (precision + recall)
#     accuracy = (TP+TN)/(TP+TN+FN+FP)

#     return accuracy
    
#     # print("final Accuracy : ",accuracy)
#     # print("final precision : ",precision)
#     # print("final recall : ",recall)
#     # print("final f1 : ",f1)

# """## word_to_vec"""

# def word_to_vec(matrix, session,config, *args):
    
#     print("word2vec shape: ", matrix.shape)
    
#     for model in args:
#         session.run(tf.compat.v1.assign(model.embedding, matrix))

# """# Main

# ## configs
# """

# argument1 = "7" # slstm iteration
# argument2 = "2" #num window_size
# argument3 = "sinhala_news" #dataset_name
# argument4 = "slstm" #model_name

# config = Config()
# config.layer=int(argument1)
# config.step=int(argument2)
# config.vocab_size=(18413) # number of words in fastText model
# print("dataset: "+argument3)
# print("iteration: "+str(config.layer))
# print("step: "+str(config.step))
# print("model: "+str(argument4))

# """## Load Embedding Matrix"""

# f = open(vector_path, 'rb')
# matrix= np.array(pickle.load(f))
# config.vocab_size=matrix.shape[0]
# print(config.vocab_size)

# """## Load Dataset"""

# # convert_to_vectors()
# # train_data_vectors, train_data_labels, test_data_vectors, test_data_labels = load_vectors()

# train_dataset, test_dataset = load_data(path=path,n_words=config.vocab_size)
# config.num_label= len(set(train_dataset[1]))

# print("number label: "+str(config.num_label))
# train_dataset = prepare_data(train_dataset[0], train_dataset[1])
# # valid_dataset = prepare_data(valid_dataset[0], valid_dataset[1])
# test_dataset = prepare_data(test_dataset[0], test_dataset[1])

# print(train_dataset)

# """## Train_Test Model"""

# with tf.Graph().as_default(), tf.compat.v1.Session() as session:
#     initializer = tf.compat.v1.random_normal_initializer(0, 0.05)

#     classifier= Classifer(config=config, session=session)
#     saver = tf.compat.v1.train.Saver()

#     total=0
#     #print trainable variables
#     # for v in tf.compat.v1.trainable_variables():
#     #     print(v.name)
#     #     shape=v.get_shape()
#     #     try:
#     #         size=shape[0].value*shape[1].value
#     #     except:
#     #         size=shape[0].value
#     #     total+=size
#     # print(total)
#     #initialize
#     init = tf.compat.v1.global_variables_initializer()

#     session.run(init)
#     #train test model

#     print ("model_test",matrix)



#     word_to_vec(matrix, session,config, classifier)
#     start_epoches(config, session,classifier, train_dataset, test_dataset)
#     # # Add ops to save and restore all the variables.
    
#     # save_path = saver.save(session, "/content/drive/My Drive/UNI/FYP/Sentiment Analysis/supportive/S-LSTM/Trained_Model/slstm_models/500epochs/1/")
#     # print("Model saved in path: %s" % save_path)

# Data Paths

In [0]:
path='/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/SLSTM/parsed_data/from_fasttext/data_set'
vector_path = '/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/SLSTM/parsed_data/from_fasttext/fasttext_vectors'
# run from lahiru1st@gmail.com
# path='/content/drive/My Drive/UNI/FYP/Sentiment Analysis/supportive/S-LSTM/classfication/parsed_data/from_fasttext/data_set'
# vector_path = '/content/drive/My Drive/UNI/FYP/Sentiment Analysis/supportive/S-LSTM/classfication/parsed_data/from_fasttext/fasttext_vectors'


# Imports

In [0]:
from __future__ import print_function
import tensorflow as tf
import pickle
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix
import numpy
import six.moves.cPickle as pickle
import time
import sys
import numpy as np
from tensorboard.plugins.hparams import api as hp
import xlwt 
from xlwt import Workbook 



In [0]:
# !pip install xlutils
# !pip install xlrd

In [0]:
from xlutils.copy import copy # http://pypi.python.org/pypi/xlutils
from xlrd import open_workbook # http://pypi.python.org/pypi/xlrd

# config

## COnfig Obj

In [0]:
class Config(object):
    vocab_size=15000
    max_grad_norm = 5
    init_scale = 0.05
    hidden_size = 300
    lr_decay = 0.95 #0.95-0.99
    valid_portion=0.0
    batch_size=8  #4-8-16-32
    keep_prob = 0.8
    #0.05
    learning_rate = 0.002
    max_epoch =2
    # max_max_epoch =40
    max_max_epoch = 15
    num_label=2
    attention_iteration=3
    random_initialize=False
    embedding_trainable=True
    l2_beta=0.0

## Set Configurations

In [0]:
# argument1 = "7" # slstm iteration
# argument2 = "2" #num window_size
# argument3 = "sinhala_news" #dataset_name
# argument4 = "slstm" #model_name

config = Config()
# config.layer=int(argument1)
# config.step=int(argument2)
# print("dataset: "+argument3)
# print("iteration: "+str(config.layer))
# print("step: "+str(config.step))
# print("model: "+str(argument4))

# Impementation

## Progress Bar

In [0]:
class Progbar(object):
    """Progbar class copied from keras (https://github.com/fchollet/keras/)

    Displays a progress bar.
    Small edit : added strict arg to update
    # Arguments
        target: Total number of steps expected.
        interval: Minimum visual progress update interval (in seconds).
    """

    def __init__(self, target, width=30, verbose=0):
        self.width = width
        self.target = target
        self.sum_values = {}
        self.unique_values = []
        self.start = time.time()
        self.total_width = 0
        self.seen_so_far = 0
        self.verbose = verbose

    def update(self, current, values=[], exact=[], strict=[]):
        """
        Updates the progress bar.
        # Arguments
            current: Index of current step.
            values: List of tuples (name, value_for_last_step).
                The progress bar will display averages for these values.
            exact: List of tuples (name, value_for_last_step).
                The progress bar will display these values directly.
        """

        for k, v in values:
            if k not in self.sum_values:
                self.sum_values[k] = [v * (current - self.seen_so_far),
                                      current - self.seen_so_far]
                self.unique_values.append(k)
            else:
                self.sum_values[k][0] += v * (current - self.seen_so_far)
                self.sum_values[k][1] += (current - self.seen_so_far)
        for k, v in exact:
            if k not in self.sum_values:
                self.unique_values.append(k)
            self.sum_values[k] = [v, 1]

        for k, v in strict:
            if k not in self.sum_values:
                self.unique_values.append(k)
            self.sum_values[k] = v

        self.seen_so_far = current

        now = time.time()
        if self.verbose == 1:
            prev_total_width = self.total_width
            sys.stdout.write("\b" * prev_total_width)
            sys.stdout.write("\r")

            numdigits = int(np.floor(np.log10(self.target))) + 1
            barstr = '%%%dd/%%%dd [' % (numdigits, numdigits)
            bar = barstr % (current, self.target)
            prog = float(current)/self.target
            prog_width = int(self.width*prog)
            if prog_width > 0:
                bar += ('='*(prog_width-1))
                if current < self.target:
                    bar += '>'
                else:
                    bar += '='
            bar += ('.'*(self.width-prog_width))
            bar += ']'
            sys.stdout.write(bar)
            self.total_width = len(bar)

            if current:
                time_per_unit = (now - self.start) / current
            else:
                time_per_unit = 0
            eta = time_per_unit*(self.target - current)
            info = ''
            if current < self.target:
                info += ' - ETA: %ds' % eta
            else:
                info += ' - %ds' % (now - self.start)
            for k in self.unique_values:
                if type(self.sum_values[k]) is list:
                    info += ' - %s: %.4f' % (k,
                        self.sum_values[k][0] / max(1, self.sum_values[k][1]))
                else:
                    info += ' - %s: %s' % (k, self.sum_values[k])

            self.total_width += len(info)
            if prev_total_width > self.total_width:
                info += ((prev_total_width-self.total_width) * " ")

            sys.stdout.write(info)
            sys.stdout.flush()

            if current >= self.target:
                sys.stdout.write("\n")

        if self.verbose == 2:
            if current >= self.target:
                info = '%ds' % (now - self.start)
                for k in self.unique_values:
                    info += ' - %s: %.4f' % (k,
                        self.sum_values[k][0] / max(1, self.sum_values[k][1]))
                sys.stdout.write(info + "\n")

    def add(self, n, values=[]):
        self.update(self.seen_so_far+n, values)


## Read Data

In [0]:
def generate_matrix(seqs, maxlen, lengths):
    n_samples = len(seqs)
    x= numpy.zeros((n_samples, maxlen)).astype('int64')

    for idx, s in enumerate(seqs):
        if lengths[idx]>= maxlen:
            s=s[:maxlen]
        x[idx, :lengths[idx]] = s
    return x

def prepare_data(seqs, labels):
    lengths = [len(s) for s in seqs]
    labels = numpy.array(labels).astype('int32')
    return [numpy.array(seqs), labels, numpy.array(lengths).astype('int32')]

def remove_unk(x, n_words):
    return [[1 if w >= n_words else w for w in sen] for sen in x] 

def load_data(path, n_words):
    with open(path, 'rb') as f:
        dataset_x, dataset_label= pickle.load(f)
        train_set_x, train_set_y = dataset_x[0], dataset_label[0]
        # valid_set_x, valid_set_y =dataset_x[1], dataset_label[1]
        test_set_x, test_set_y = dataset_x[1], dataset_label[1]
    #remove unknown words
    train_set_x = remove_unk(train_set_x, n_words)
    # valid_set_x = remove_unk(valid_set_x, n_words)
    test_set_x = remove_unk(test_set_x, n_words)

    return [train_set_x, train_set_y],[test_set_x, test_set_y]

## SLSTM layer

In [0]:
def lstm_layer(initial_hidden_states, config, keep_prob, mask):
    with tf.compat.v1.variable_scope('forward'):
        fw_lstm = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(config.hidden_size, forget_bias=0.0)
        fw_lstm = tf.contrib.rnn.DropoutWrapper(fw_lstm, output_keep_prob=keep_prob)

    with tf.compat.v1.variable_scope('backward'):
        bw_lstm = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(config.hidden_size, forget_bias=0.0)
        bw_lstm = tf.contrib.rnn.DropoutWrapper(bw_lstm, output_keep_prob=keep_prob)

    #bidirectional rnn
    with tf.compat.v1.variable_scope('bilstm'):
        lstm_output=tf.compat.v1.nn.bidirectional_dynamic_rnn(fw_lstm, bw_lstm, inputs=initial_hidden_states, sequence_length=mask, time_major=False, dtype=tf.float32)
        lstm_output=tf.concat(lstm_output[0], 2)

    return lstm_output


## Classifier

In [0]:
class Classifer(object):

    def get_hidden_states_before(self, hidden_states, step, shape, hidden_size):
        #padding zeros
        padding=tf.zeros((shape[0], step, hidden_size), dtype=tf.float32)
        #remove last steps
        displaced_hidden_states=hidden_states[:,:-step,:]
        #concat padding
        return tf.concat([padding, displaced_hidden_states], axis=1)
        #return tf.cond(step<=shape[1], lambda: tf.concat([padding, displaced_hidden_states], axis=1), lambda: tf.zeros((shape[0], shape[1], self.config.hidden_size_sum), dtype=tf.float32))

    def get_hidden_states_after(self, hidden_states, step, shape, hidden_size):
        #padding zeros
        padding=tf.zeros((shape[0], step, hidden_size), dtype=tf.float32)
        #remove last steps
        displaced_hidden_states=hidden_states[:,step:,:]
        #concat padding
        return tf.concat([displaced_hidden_states, padding], axis=1)
        #return tf.cond(step<=shape[1], lambda: tf.concat([displaced_hidden_states, padding], axis=1), lambda: tf.zeros((shape[0], shape[1], self.config.hidden_size_sum), dtype=tf.float32))

    def sum_together(self, l):
        combined_state=None
        for tensor in l:
            if combined_state==None:
                combined_state=tensor
            else:
                combined_state=combined_state+tensor
        return combined_state
    
    def slstm_cell(self, name_scope_name, hidden_size, lengths, initial_hidden_states, initial_cell_states, num_layers):
        with tf.compat.v1.name_scope(name_scope_name):
            #Word parameters 
            #forget gate for left 
            with tf.compat.v1.name_scope("f1_gate"):
                #current
                Wxf1 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
                #left right
                Whf1 = tf.Variable(tf.random.normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
                #initial state
                Wif1 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wif")
                #dummy node
                Wdf1 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdf")
            #forget gate for right 
            with tf.compat.v1.name_scope("f2_gate"):
                Wxf2 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
                Whf2 = tf.Variable(tf.random.normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
                Wif2 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wif")
                Wdf2 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdf")
            #forget gate for inital states     
            with tf.compat.v1.name_scope("f3_gate"):
                Wxf3 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
                Whf3 = tf.Variable(tf.random.normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
                Wif3 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wif")
                Wdf3 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdf")
            #forget gate for dummy states     
            with tf.compat.v1.name_scope("f4_gate"):
                Wxf4 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
                Whf4 = tf.Variable(tf.random.normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
                Wif4 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wif")
                Wdf4 = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdf")
            #input gate for current state     
            with tf.compat.v1.name_scope("i_gate"):
                Wxi = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxi")
                Whi = tf.Variable(tf.random.normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whi")
                Wii = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wii")
                Wdi = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdi")
            #input gate for output gate
            with tf.compat.v1.name_scope("o_gate"):
                Wxo = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxo")
                Who = tf.Variable(tf.random.normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Who")
                Wio = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wio")
                Wdo = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdo")
            #bias for the gates    
            with tf.compat.v1.name_scope("biases"):
                bi = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bi")
                bo = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bo")
                bf1 = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bf1")
                bf2 = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bf2")
                bf3 = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bf3")
                bf4 = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bf4")

            #dummy node gated attention parameters
            #input gate for dummy state
            with tf.compat.v1.name_scope("gated_d_gate"):
                gated_Wxd = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
                gated_Whd = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
            #output gate
            with tf.compat.v1.name_scope("gated_o_gate"):
                gated_Wxo = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxo")
                gated_Who = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Who")
            #forget gate for states of word
            with tf.compat.v1.name_scope("gated_f_gate"):
                gated_Wxf = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxo")
                gated_Whf = tf.Variable(tf.random.normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Who")
            #biases
            with tf.compat.v1.name_scope("gated_biases"):
                gated_bd = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bi")
                gated_bo = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bo")
                gated_bf = tf.Variable(tf.random.normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bo")

        #filters for attention        
        mask_softmax_score=tf.cast(tf.sequence_mask(lengths), tf.float32)*1e25-1e25
        mask_softmax_score_expanded=tf.expand_dims(mask_softmax_score, axis=2)               
        #filter invalid steps
        sequence_mask=tf.expand_dims(tf.cast(tf.sequence_mask(lengths), tf.float32),axis=2)
        #filter embedding states
        initial_hidden_states=initial_hidden_states*sequence_mask
        initial_cell_states=initial_cell_states*sequence_mask
        #record shape of the batch
        shape=tf.shape(input=initial_hidden_states)
        
        #initial embedding states
        embedding_hidden_state=tf.reshape(initial_hidden_states, [-1, hidden_size])      
        embedding_cell_state=tf.reshape(initial_cell_states, [-1, hidden_size])

        #randomly initialize the states
        if config.random_initialize:
            initial_hidden_states=tf.random.uniform(shape, minval=-0.05, maxval=0.05, dtype=tf.float32, seed=None, name=None)
            initial_cell_states=tf.random.uniform(shape, minval=-0.05, maxval=0.05, dtype=tf.float32, seed=None, name=None)
            #filter it
            initial_hidden_states=initial_hidden_states*sequence_mask
            initial_cell_states=initial_cell_states*sequence_mask

        #inital dummy node states
        dummynode_hidden_states=tf.reduce_mean(input_tensor=initial_hidden_states, axis=1)
        dummynode_cell_states=tf.reduce_mean(input_tensor=initial_cell_states, axis=1)

        for i in range(num_layers):
            #update dummy node states
            #average states
            combined_word_hidden_state=tf.reduce_mean(input_tensor=initial_hidden_states, axis=1)
            reshaped_hidden_output=tf.reshape(initial_hidden_states, [-1, hidden_size])
            #copy dummy states for computing forget gate
            transformed_dummynode_hidden_states=tf.reshape(tf.tile(tf.expand_dims(dummynode_hidden_states, axis=1), [1, shape[1],1]), [-1, hidden_size])
            #input gate
            gated_d_t = tf.nn.sigmoid(
                tf.matmul(dummynode_hidden_states, gated_Wxd) + tf.matmul(combined_word_hidden_state, gated_Whd) + gated_bd
            )
            #output gate
            gated_o_t = tf.nn.sigmoid(
                tf.matmul(dummynode_hidden_states, gated_Wxo) + tf.matmul(combined_word_hidden_state, gated_Who) + gated_bo
            )
            #forget gate for hidden states
            gated_f_t = tf.nn.sigmoid(
                tf.matmul(transformed_dummynode_hidden_states, gated_Wxf) + tf.matmul(reshaped_hidden_output, gated_Whf) + gated_bf
            )

            #softmax on each hidden dimension 
            reshaped_gated_f_t=tf.reshape(gated_f_t, [shape[0], shape[1], hidden_size])+ mask_softmax_score_expanded
            gated_softmax_scores=tf.nn.softmax(tf.concat([reshaped_gated_f_t, tf.expand_dims(gated_d_t, axis=1)], axis=1), axis=1)
            #split the softmax scores
            new_reshaped_gated_f_t=gated_softmax_scores[:,:shape[1],:]
            new_gated_d_t=gated_softmax_scores[:,shape[1]:,:]
            #new dummy states
            dummy_c_t=tf.reduce_sum(input_tensor=new_reshaped_gated_f_t * initial_cell_states, axis=1) + tf.squeeze(new_gated_d_t, axis=1)*dummynode_cell_states
            dummy_h_t=gated_o_t * tf.nn.tanh(dummy_c_t)

            #update word node states
            #get states before
            initial_hidden_states_before=[tf.reshape(self.get_hidden_states_before(initial_hidden_states, step+1, shape, hidden_size), [-1, hidden_size]) for step in range(self.config.step)]
            initial_hidden_states_before=self.sum_together(initial_hidden_states_before)
            initial_hidden_states_after= [tf.reshape(self.get_hidden_states_after(initial_hidden_states, step+1, shape, hidden_size), [-1, hidden_size]) for step in range(self.config.step)]
            initial_hidden_states_after=self.sum_together(initial_hidden_states_after)
            #get states after
            initial_cell_states_before=[tf.reshape(self.get_hidden_states_before(initial_cell_states, step+1, shape, hidden_size), [-1, hidden_size]) for step in range(self.config.step)]
            initial_cell_states_before=self.sum_together(initial_cell_states_before)
            initial_cell_states_after=[tf.reshape(self.get_hidden_states_after(initial_cell_states, step+1, shape, hidden_size), [-1, hidden_size]) for step in range(self.config.step)]
            initial_cell_states_after=self.sum_together(initial_cell_states_after)
            
            #reshape for matmul
            initial_hidden_states=tf.reshape(initial_hidden_states, [-1, hidden_size])
            initial_cell_states=tf.reshape(initial_cell_states, [-1, hidden_size])

            #concat before and after hidden states
            concat_before_after=tf.concat([initial_hidden_states_before, initial_hidden_states_after], axis=1)

            #copy dummy node states 
            transformed_dummynode_hidden_states=tf.reshape(tf.tile(tf.expand_dims(dummynode_hidden_states, axis=1), [1, shape[1],1]), [-1, hidden_size])
            transformed_dummynode_cell_states=tf.reshape(tf.tile(tf.expand_dims(dummynode_cell_states, axis=1), [1, shape[1],1]), [-1, hidden_size])

            f1_t = tf.nn.sigmoid(
                tf.matmul(initial_hidden_states, Wxf1) + tf.matmul(concat_before_after, Whf1) + 
                tf.matmul(embedding_hidden_state, Wif1) + tf.matmul(transformed_dummynode_hidden_states, Wdf1)+ bf1
            )

            f2_t = tf.nn.sigmoid(
                tf.matmul(initial_hidden_states, Wxf2) + tf.matmul(concat_before_after, Whf2) + 
                tf.matmul(embedding_hidden_state, Wif2) + tf.matmul(transformed_dummynode_hidden_states, Wdf2)+ bf2
            )

            f3_t = tf.nn.sigmoid(
                tf.matmul(initial_hidden_states, Wxf3) + tf.matmul(concat_before_after, Whf3) + 
                tf.matmul(embedding_hidden_state, Wif3) + tf.matmul(transformed_dummynode_hidden_states, Wdf3) + bf3
            )

            f4_t = tf.nn.sigmoid(
                tf.matmul(initial_hidden_states, Wxf4) + tf.matmul(concat_before_after, Whf4) + 
                tf.matmul(embedding_hidden_state, Wif4) + tf.matmul(transformed_dummynode_hidden_states, Wdf4) + bf4
            )
            
            i_t = tf.nn.sigmoid(
                tf.matmul(initial_hidden_states, Wxi) + tf.matmul(concat_before_after, Whi) + 
                tf.matmul(embedding_hidden_state, Wii) + tf.matmul(transformed_dummynode_hidden_states, Wdi)+ bi
            )
            
            o_t = tf.nn.sigmoid(
                tf.matmul(initial_hidden_states, Wxo) + tf.matmul(concat_before_after, Who) + 
                tf.matmul(embedding_hidden_state, Wio) + tf.matmul(transformed_dummynode_hidden_states, Wdo) + bo
            )
            
            f1_t, f2_t, f3_t, f4_t, i_t=tf.expand_dims(f1_t, axis=1), tf.expand_dims(f2_t, axis=1),tf.expand_dims(f3_t, axis=1), tf.expand_dims(f4_t, axis=1), tf.expand_dims(i_t, axis=1)


            five_gates=tf.concat([f1_t, f2_t, f3_t, f4_t,i_t], axis=1)
            five_gates=tf.nn.softmax(five_gates, axis=1)
            f1_t,f2_t,f3_t, f4_t,i_t= tf.split(five_gates, num_or_size_splits=5, axis=1)
            
            f1_t, f2_t, f3_t, f4_t, i_t=tf.squeeze(f1_t, axis=1), tf.squeeze(f2_t, axis=1),tf.squeeze(f3_t, axis=1), tf.squeeze(f4_t, axis=1),tf.squeeze(i_t, axis=1)

            c_t = (f1_t * initial_cell_states_before) + (f2_t * initial_cell_states_after)+(f3_t * embedding_cell_state)+ (f4_t * transformed_dummynode_cell_states)+ (i_t * initial_cell_states)
            
            h_t = o_t * tf.nn.tanh(c_t)

            #update states
            initial_hidden_states=tf.reshape(h_t, [shape[0], shape[1], hidden_size])
            initial_cell_states=tf.reshape(c_t, [shape[0], shape[1], hidden_size])
            initial_hidden_states=initial_hidden_states*sequence_mask
            initial_cell_states=initial_cell_states*sequence_mask

            dummynode_hidden_states=dummy_h_t
            dummynode_cell_states=dummy_c_t

        initial_hidden_states = tf.nn.dropout(initial_hidden_states,1 - (self.dropout))
        initial_cell_states = tf.nn.dropout(initial_cell_states, 1 - (self.dropout))

        return initial_hidden_states, initial_cell_states, dummynode_hidden_states



    def __init__(self, config, session):
        #inputs: features, mask, keep_prob, labels
        self.input_data = tf.compat.v1.placeholder(tf.int32, [None, None], name="inputs")
        self.labels=tf.compat.v1.placeholder(tf.int64, [None,], name="labels")
        self.mask=tf.compat.v1.placeholder(tf.int32, [None,], name="mask")
        self.dropout=self.keep_prob=keep_prob=tf.compat.v1.placeholder(tf.float32, name="keep_prob")
        self.config=config
        shape=tf.shape(input=self.input_data)
        #if sys.argv[4]=='lstm':
        #    self.dummy_input = tf.placeholder(tf.float32, [None, None], name="dummy")
        #embedding
        self.embedding=embedding = tf.Variable(tf.random.normal([config.vocab_size, config.hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="embedding", trainable=config.embedding_trainable)
        #apply embedding
        initial_hidden_states=tf.nn.embedding_lookup(params=embedding, ids=self.input_data)
        initial_cell_states=tf.identity(initial_hidden_states)

        initial_hidden_states = tf.nn.dropout(initial_hidden_states,1 - (keep_prob))
        initial_cell_states = tf.nn.dropout(initial_cell_states, 1 - (keep_prob))

        #create layers 
        if config.model_type=='slstm':
            new_hidden_states,new_cell_state, dummynode_hidden_states=self.slstm_cell("word_slstm", config.hidden_size,self.mask, initial_hidden_states, initial_cell_states, config.layer)
            
            softmax_w = tf.Variable(tf.random.normal([2*config.hidden_size, config.num_label], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_w")
            softmax_b = tf.Variable(tf.random.normal([config.num_label], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_b")
            #representation=dummynode_hidden_states
            representation=tf.reduce_mean(input_tensor=tf.concat([new_hidden_states, tf.expand_dims(dummynode_hidden_states, axis=1)], axis=1), axis=1)
            
            softmax_w2 = tf.Variable(tf.random.normal([config.hidden_size, 2*config.hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_w2")
            softmax_b2 = tf.Variable(tf.random.normal([2*config.hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_b2")
            representation=tf.nn.tanh(tf.matmul(representation, softmax_w2)+softmax_b2)

        elif config.model_type=='lstm':
            initial_hidden_states=lstm_layer(initial_hidden_states,config, self.keep_prob, self.mask)
            softmax_w = tf.Variable(tf.random.normal([2*config.hidden_size, config.num_label], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_w")
            softmax_b = tf.Variable(tf.random.normal([config.num_label], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_b")
            representation=tf.reduce_sum(input_tensor=initial_hidden_states,axis=1)
            config.hidden_size_sum=2*config.hidden_size
        elif config.model_type=='cnn':
            initial_hidden_states=tf.reshape(initial_hidden_states, [-1, 700, config.hidden_size])            
            initial_hidden_states = tf.expand_dims(initial_hidden_states, -1)
            pooled_outputs = []
            for i, filter_size in enumerate([3]):
                with tf.compat.v1.name_scope("conv-maxpool-%s" % filter_size):
                    # Convolution Layer
                    filter_shape = [filter_size, config.hidden_size, 1, config.hidden_size]
                    W = tf.Variable(tf.random.truncated_normal(filter_shape, stddev=0.1), name="W")
                    b = tf.Variable(tf.constant(0.1, shape=[config.hidden_size]), name="b")

                    W2 = tf.Variable(tf.random.truncated_normal(filter_shape, stddev=0.1), name="W2")
                    b2 = tf.Variable(tf.constant(0.1, shape=[config.hidden_size]), name="b2")

                    W3 = tf.Variable(tf.random.truncated_normal(filter_shape, stddev=0.1), name="W3")
                    b3 = tf.Variable(tf.constant(0.1, shape=[config.hidden_size]), name="b3")

                    W4 = tf.Variable(tf.random.truncated_normal(filter_shape, stddev=0.1), name="W4")
                    b4 = tf.Variable(tf.constant(0.1, shape=[config.hidden_size]), name="b4")

                    conv = tf.nn.conv2d(
                        input=initial_hidden_states,
                        filters=W,
                        strides=[1, 1, 1, 1],
                        padding="VALID",
                        name="conv")
                    h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                    print(h.get_shape())
                    h=tf.transpose(a=h, perm=[0,1,3,2])
                    # Apply nonlinearity


                    conv2 = tf.nn.conv2d(
                        input=h,
                        filters=W2,
                        strides=[1, 1, 1, 1],
                        padding="VALID",
                        name="conv2")
                    h2 = tf.nn.relu(tf.nn.bias_add(conv2, b2), name="relu2")
                    print(h2.get_shape())
                    h2=tf.transpose(a=h2, perm=[0,1,3,2])

                    conv3 = tf.nn.conv2d(
                        input=h2,
                        filters=W3,
                        strides=[1, 1, 1, 1],
                        padding="VALID",
                        name="conv3")  
                    h3 = tf.nn.relu(tf.nn.bias_add(conv3, b3), name="relu3")
                    print(h3.get_shape())

                    # Max-pooling over the outputs
                    pooled = tf.nn.max_pool2d(
                        input=h3,
                        ksize=[1, 700 - 3*filter_size + 3, 1, 1],
                        strides=[1, 1, 1, 1],
                        padding='VALID',
                        name="pool")
                    pooled_outputs.append(pooled)
            # Combine all the pooled features
            num_filters_total = 1 * config.hidden_size
            self.h_pool = tf.concat(pooled_outputs, axis=3)
            representation = tf.reshape(self.h_pool, [-1, num_filters_total])

            softmax_w = tf.Variable(tf.random.normal([2*config.hidden_size, config.num_label], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_w")
            softmax_b = tf.Variable(tf.random.normal([config.num_label], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_b")

            softmax_w2 = tf.Variable(tf.random.normal([config.hidden_size, 2*config.hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_w2")
            softmax_b2 = tf.Variable(tf.random.normal([2*config.hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_b2")
            representation=tf.nn.tanh(tf.matmul(representation, softmax_w2)+softmax_b2)
        else:
            print("Invalid model")
            exit(1)
        
        self.logits=logits = tf.matmul(representation, softmax_w) + softmax_b
        self.to_print=tf.nn.softmax(logits)
        #operators for prediction
        self.prediction=prediction=tf.argmax(input=logits,axis=1)
        correct_prediction = tf.equal(prediction, self.labels)
        self.accuracy = tf.reduce_sum(input_tensor=tf.cast(correct_prediction, tf.float32))
        
        #cross entropy loss
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.labels, logits=logits)
        self.cost=cost=tf.reduce_mean(input_tensor=loss)+ config.l2_beta*tf.nn.l2_loss(embedding)

        #designate training variables
        tvars=tf.compat.v1.trainable_variables()
        self.lr = tf.Variable(0.0, trainable=False)
        grads=tf.gradients(ys=cost, xs=tvars)
        grads, _ = tf.clip_by_global_norm(grads,config.max_grad_norm)
        self.grads=grads
        optimizer = tf.compat.v1.train.AdamOptimizer(config.learning_rate)        
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))

    #assign value to learning rate
    def assign_lr(self, session, lr_value):
        session.run(tf.compat.v1.assign(self.lr, lr_value))

## Train_Test Implementation

In [0]:
def get_minibatches_idx(n, batch_size, shuffle=True):
    idx_list = np.arange(n, dtype="int32")

    if shuffle:
        np.random.shuffle(idx_list)

    minibatches = []
    minibatch_start = 0
    for i in range(n // batch_size):
        minibatches.append(idx_list[minibatch_start:
                                    minibatch_start + batch_size])
        minibatch_start += batch_size
    if (minibatch_start != n):
        # Make a minibatch out of what is left
        minibatches.append(idx_list[minibatch_start:])
    return minibatches

"""## run_epoch"""

def run_epoch(session, config, model, data, eval_op, keep_prob, is_training):
    n_samples = len(data[0])
    # print("Running %d samples:"%(n_samples))  
    minibatches = get_minibatches_idx(n_samples, config.batch_size, shuffle=False)

    predictions = []
    correct = 0.
    total = 0
    total_cost=0
    prog = Progbar(target=len(minibatches))
    #dummynode_hidden_states_collector=np.array([[0]*config.hidden_size])

    to_print_total=np.array([[0]*2])
    for i, inds in enumerate(minibatches):
        x = data[0][inds]
        if config.model_type=='cnn':
            x=pad_sequences(x, maxlen=700, dtype='int32',padding='post', truncating='post', value=0.)
        else:
            x=pad_sequences(x, maxlen=None, dtype='int32',padding='post', truncating='post', value=0.)
        y = data[1][inds]
        mask = data[2][inds]



        count, _, cost, to_print,prediction= \
        session.run([model.accuracy, eval_op,model.cost, model.to_print,model.prediction],\
            {model.input_data: x, model.labels: y, model.mask:mask, model.keep_prob:keep_prob}) 
        

        if not is_training:
            to_print_total=np.concatenate((to_print_total, to_print),axis=0)
        correct += count 
        total += len(inds)
        total_cost+=cost
        predictions.extend(prediction.tolist())
        prog.update(i + 1, [("train loss", cost)])
    #if not is_training:
    #    print(to_print_total[:, 0].tolist())
    #    print(data[1].tolist())
    #    print(data[2].tolist())

    actual = data[1]

    TN, FP, FN, TP = confusion_matrix(actual, predictions).ravel()

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precision * recall / (precision + recall)

    print("Total loss: ",total_cost)

    accuracy = correct/total

    return accuracy,precision,recall,f1, actual, predictions

"""## train_test_model"""

def train_model(config, i, session, model, train_dataset):
    #compute lr_decay
    lr_decay = config.lr_decay ** max(i - config.max_epoch, 0.0)
    #update learning rate
    model.assign_lr(session, config.learning_rate * lr_decay)

    #training            
    # print("Epoch: %d Learning rate: %.5f" % (i + 1, session.run(model.lr)))
    start_time = time.time()
    train_acc, train_precision, train_recall, train_f1,actual,predictions = run_epoch(session, config, model, train_dataset, model.train_op, config.keep_prob, True)
    print("Training Accuracy = %.4f, time = %.3f seconds\n"%(train_acc, time.time()-start_time))

    # #valid 
    # valid_acc = run_epoch(session, config, model, valid_dataset, tf.no_op(),1, False)
    # print("Valid Accuracy = %.4f\n" % valid_acc)

    #testing
    # start_time = time.time()
    # test_acc,test_precision, test_recall, test_f1,test_actual,test_predictions = run_epoch(session, config, model, test_dataset, tf.no_op(),1, False)
    # print("Test Accuracy = %.4f, Test Precision = %.4f, Test Recall = %.4f, Test F1 = %.4f\n" % (test_acc,test_precision,test_recall,test_f1))    
    # print("Time = %.3f seconds\n"%(time.time()-start_time))
    # print('confusion metric : ')
    # print((confusion_matrix(test_actual, test_predictions).ravel()))

    #return valid_acc, test_acc
    # return test_actual,test_predictions

"""## start_epoches"""

def start_epoches(config, session,classifier, train_dataset):
    #record max
    #max_val_acc=-1
    #max_test_acc=-1
    # saver = tf.train.Saver()
    # all_actual = []
    # all_predictions = []

    for i in range(config.max_max_epoch):
      train_model(config, i, session, classifier, train_dataset)
      
      # all_actual.extend(test_actual)
      # all_predictions.extend(test_predictions)


      # if (i % 1 == 0 and i != 0):
      #   save_path = saver.save(session, "/content/drive/My Drive/University/FYP/Sentiment Analysis/supportive/S-LSTM/Trained_Model/slstm_models/pretrained_lstm.ckpt", global_step=i)
      #     print("saved to %s" % save_path)

    # TN, FP, FN, TP = confusion_matrix(all_actual, all_predictions).ravel()

    # precision = TP / (TP + FP)
    # recall = TP / (TP + FN)
    # f1 = 2 * precision * recall / (precision + recall)
    # accuracy = (TP+TN)/(TP+TN+FN+FP)

    # return accuracy
    
    # print("final Accuracy : ",accuracy)
    # print("final precision : ",precision)
    # print("final recall : ",recall)
    # print("final f1 : ",f1)

"""## word_to_vec"""

def word_to_vec(matrix, session,config, *args):
    
    print("word2vec shape: ", matrix.shape)
    
    for model in args:
        session.run(tf.compat.v1.assign(model.embedding, matrix))

# Main

## Load Dataset and Embedding

In [151]:

f = open(vector_path, 'rb')
matrix= np.array(pickle.load(f))
config.vocab_size=matrix.shape[0]
# print(config.vocab_size)

"""## Load Dataset"""

# convert_to_vectors()
# train_data_vectors, train_data_labels, test_data_vectors, test_data_labels = load_vectors()

train_dataset, test_dataset = load_data(path=path,n_words=config.vocab_size)
config.num_label= len(set(train_dataset[1]))

# print("number label: "+str(config.num_label))
train_dataset = prepare_data(train_dataset[0], train_dataset[1])
# valid_dataset = prepare_data(valid_dataset[0], valid_dataset[1])
test_dataset = prepare_data(test_dataset[0], test_dataset[1])

print(train_dataset)

[array([list([133, 105, 445, 19, 207, 2, 6, 4429, 12, 4430, 1533, 375, 77, 160, 33, 7, 10, 423, 17, 2088, 155, 10, 3205]),
       list([204, 26]),
       list([318, 180, 4431, 303, 7096, 181, 2512, 7097, 7, 4432, 682, 7098, 723, 48, 7099, 7100, 2513, 4433, 1534, 887, 772, 7101, 7102, 1337, 7103, 1338, 7104, 7105, 34, 150, 127]),
       ...,
       list([5173, 8, 2340, 3150, 22, 297, 76, 6640, 16198, 420, 3586, 16199, 1531, 164, 16200, 1932, 625, 16201, 469, 133, 1980, 4291, 218, 5065, 16202]),
       list([1802, 6909, 716, 5437, 2938, 6613, 350, 16203, 6843, 1802, 5440, 395, 78]),
       list([329, 7, 1043, 64, 2180])], dtype=object), array([1, 0, 1, ..., 0, 0, 1], dtype=int32), array([23,  2, 31, ..., 25, 13,  5], dtype=int32)]


## Train and Test Model

In [152]:
!rm -rf ./logs/ 
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
def main(run_dir,hparams,config):
  with tf.Graph().as_default(),tf.compat.v1.Session() as session:

      initializer = tf.compat.v1.random_normal_initializer(0, 0.05)

      classifier= Classifer(config=config, session=session)
      # saver = tf.compat.v1.train.Saver()

      # total=0
      # #print trainable variables
      # for v in tf.compat.v1.trainable_variables():
      #     print(v.name)
      #     shape=v.get_shape()
      #     try:
      #         size=shape[0].value*shape[1].value
      #     except:
      #         size=shape[0].value
      #     total+=size
      # print(total)
      #initialize
      init = tf.compat.v1.global_variables_initializer()

      session.run(init)
      #train test model

      # print ("model_test",matrix)



      word_to_vec(matrix, session,config, classifier)
      start_epoches(config, session,classifier, train_dataset)

      test_acc,test_precision, test_recall, test_f1,test_actual,test_predictions = run_epoch(session, config, classifier, test_dataset, tf.no_op(),1, False)
      print("Test Accuracy = %.4f, Test Precision = %.4f, Test Recall = %.4f, Test F1 = %.4f\n" % (test_acc,test_precision,test_recall,test_f1))
      
      with tf.summary.create_file_writer(run_dir).as_default():
        print("Run directory is : ",run_dir)
        hp.hparams(hparams)
        tf.summary.scalar(METRIC_ACCURACY, test_acc, step=1)
      return test_acc
      # # Add ops to save and restore all the variables.
      
      # save_path = saver.save(session, "/content/drive/My Drive/UNI/FYP/Sentiment Analysis/supportive/S-LSTM/Trained_Model/slstm_models/500epochs/1/")
      # print("Model saved in path: %s" % save_path)

# HParams Tuning

In [0]:
HP_LR_DECAY = hp.HParam('lr_decay', hp.RealInterval(0.95,0.99))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.00001, 0.00002, 0.00003, 0.00004,0.00005]))
# HP_EMBEDDING_TRAINABLE = hp.HParam('embedding_trainable', hp.Discrete([True, False]))
# HP_RANDOM_INITIALIZE = hp.HParam('random_initialize',hp.Discrete([True]))
# HP_ATTENTION_ITERATION = hp.HParam('attention_iteration',hp.IntInterval(3))
# HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([8]))
# HP_KEEP_PROB = hp.HParam('keep_prob', hp.Discrete([0.8]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning/').as_default():
  hp.hparams_config(
    hparams=[
             HP_LEARNING_RATE,
              HP_LR_DECAY
            #  , HP_EMBEDDING_TRAINABLE,HP_RANDOM_INITIALIZE,HP_ATTENTION_ITERATION
             ],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [0]:
# for learning_rate in tf.linspace(HP_ATTENTION_ITERATION.domain.min_value,HP_ATTENTION_ITERATION.domain.max_value,1):
#   print(learning_rate)

In [0]:
def run(run_dir, hparams,config):
  argument1 = "7" # slstm iteration
  argument2 = "2" #num window_size
  argument3 = "sinhala_news" #dataset_name
  argument4 = "slstm" #model_name
  hparamsNew = hparams
  # config = Config()
  config.layer=int(argument1)
  config.step=int(argument2)
  config.max_max_epoch=30
  config.model_type = argument4
  config.learning_rate=hparams[HP_LEARNING_RATE]
  config.lr_decay=hparams[HP_LR_DECAY]
  # config.embedding_trainable=hparams[HP_EMBEDDING_TRAINABLE]
  # config.random_initialize=hparams[HP_RANDOM_INITIALIZE]
  # config.attention_iteration=hparams[HP_ATTENTION_ITERATION]
  # config.batch_size=hparams[HP_BATCH_SIZE]
  # config.keep_prob=hparams[HP_KEEP_PROB]

  accuracy = main(run_dir,hparamsNew,config)
  return accuracy
  


In [157]:
session_num = 12
file_path = '/content/drive/My Drive/University/FYP/Sentiment Analysis/Reports/hparams_results_1.xlsx'
rb = open_workbook(file_path)
r_sheet = rb.sheet_by_index(2) # read only copy to introspect the file
wb = copy(rb) # a writable copy (I can't read values out of this, only write to it)
sheet1 = wb.get_sheet(2) # the sheet to write to within the writable copy
sheet1.write(0, 0, 'learning_rate') 
sheet1.write(0, 1, 'lr_decay') 
sheet1.write(0, 2, 'random_nitialize') 
sheet1.write(0, 3, 'attention_iteration') 
sheet1.write(0, 4, 'batch_size')
sheet1.write(0, 5, 'keep_prob')

# lr_decay = config.lr_decay
random_initialize = config.random_initialize
attention_iteration = config.attention_iteration
batch_size = config.batch_size
keep_prob = config.keep_prob

sheet1.write(0, 6, 'accuracy')
for learning_rate in HP_LEARNING_RATE.domain.values:
  for lr_decay in (HP_LR_DECAY.domain.min_value, HP_LR_DECAY.domain.max_value):
  #   for random_initialize in HP_RANDOM_INITIALIZE.domain.values:
  #     for attention_iteration in (HP_ATTENTION_ITERATION .domain.min_value, HP_ATTENTION_ITERATION.domain.max_value):
  #       for batch_size in HP_BATCH_SIZE.domain.values:
  #         for keep_prob in HP_KEEP_PROB.domain.values:
    hparams = {
    HP_LEARNING_RATE: learning_rate,
    HP_LR_DECAY: lr_decay,
    # # HP_EMBEDDING_TRAINABLE: embedding_trainable,
    # HP_RANDOM_INITIALIZE : random_initialize,
    # HP_ATTENTION_ITERATION : attention_iteration,
    # HP_BATCH_SIZE:batch_size,
    # HP_KEEP_PROB:keep_prob
    }
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    accuracy = run('logs/hparam_tuning/' + run_name, hparams,config)
    session_num += 1

    sheet1.write(session_num+1, 0, learning_rate) 
    sheet1.write(session_num+1, 1, lr_decay) 
    sheet1.write(session_num+1, 2, random_initialize) 
    sheet1.write(session_num+1, 3, attention_iteration) 
    sheet1.write(session_num+1, 4, batch_size)
    sheet1.write(session_num+1, 5, keep_prob)
    sheet1.write(session_num+1, 6, accuracy )
    wb.save(file_path)





--- Starting trial: run-12
{'learning_rate': 1e-05, 'lr_decay': 0.95}
word2vec shape:  (18415, 300)
Total loss:  321.9241454601288
Training Accuracy = 0.5903, time = 20.035 seconds

Total loss:  280.8529450595379
Training Accuracy = 0.7420, time = 16.541 seconds

Total loss:  257.9502979964018
Training Accuracy = 0.8061, time = 16.213 seconds

Total loss:  242.83408145606518
Training Accuracy = 0.8086, time = 16.124 seconds

Total loss:  229.93427188694477
Training Accuracy = 0.8139, time = 16.305 seconds

Total loss:  219.6506710499525
Training Accuracy = 0.8204, time = 16.166 seconds

Total loss:  212.80911703407764
Training Accuracy = 0.8293, time = 16.089 seconds

Total loss:  205.14210444688797
Training Accuracy = 0.8371, time = 16.080 seconds

Total loss:  199.4897791221738
Training Accuracy = 0.8398, time = 16.220 seconds

Total loss:  193.59031739085913
Training Accuracy = 0.8441, time = 16.077 seconds

Total loss:  187.93729720264673
Training Accuracy = 0.8520, time = 16.008 s